In [1]:
import numpy as np
import pandas as pd
import random 
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_excel('Online Retail.xlsx')

In [3]:
dataset['StockCode'] = dataset['StockCode'].astype(str)

In [4]:
dataset.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [30]:
dataset.dropna(inplace=True)

In [31]:
len(dataset['CustomerID'].unique())

4372

In [32]:
customers = (dataset['CustomerID'].unique()).tolist()


In [33]:
customers

[17850.0,
 13047.0,
 12583.0,
 13748.0,
 15100.0,
 15291.0,
 14688.0,
 17809.0,
 15311.0,
 14527.0,
 16098.0,
 18074.0,
 17420.0,
 16029.0,
 16250.0,
 12431.0,
 17511.0,
 17548.0,
 13705.0,
 13747.0,
 13408.0,
 13767.0,
 17924.0,
 13448.0,
 15862.0,
 15513.0,
 12791.0,
 16218.0,
 14045.0,
 14307.0,
 17908.0,
 17920.0,
 12838.0,
 13255.0,
 16583.0,
 18085.0,
 13758.0,
 13694.0,
 15983.0,
 14849.0,
 17968.0,
 16210.0,
 17897.0,
 17377.0,
 16552.0,
 17181.0,
 17951.0,
 14729.0,
 12748.0,
 15012.0,
 12868.0,
 17572.0,
 14078.0,
 14001.0,
 12662.0,
 15525.0,
 14237.0,
 17905.0,
 15485.0,
 12433.0,
 16955.0,
 15350.0,
 15605.0,
 18144.0,
 15922.0,
 14594.0,
 15165.0,
 14911.0,
 16456.0,
 17841.0,
 12472.0,
 17346.0,
 17643.0,
 17873.0,
 13093.0,
 12921.0,
 13468.0,
 17760.0,
 16928.0,
 16048.0,
 16274.0,
 14496.0,
 14696.0,
 16539.0,
 17025.0,
 13777.0,
 17690.0,
 12947.0,
 17460.0,
 18229.0,
 14142.0,
 17069.0,
 13065.0,
 14606.0,
 16835.0,
 15235.0,
 13576.0,
 18011.0,
 13090.0,
 15694.0,


In [34]:
random.shuffle(customers)

In [35]:
purchase = []

In [36]:
from tqdm import tqdm

In [37]:
purchase = []

for i in tqdm(customers):
  temp = dataset[dataset['CustomerID']==i]['StockCode'].tolist()
  purchase.append(temp)

100%|██████████| 4372/4372 [00:05<00:00, 802.49it/s]


In [38]:
len(purchase)

4372

In [39]:
random.shuffle(purchase)

In [40]:
purchase_train = purchase[ : 3497]

In [41]:
purchase_test = purchase[ 3497:]

In [43]:
from gensim.models import Word2Vec

In [45]:
model = Word2Vec(window=10 , sg=1 , hs=0 , negative=10 , alpha=0.03  ,min_alpha=0.0007 , seed=14)

In [46]:
model.build_vocab(purchase_train , progress_per=200)

In [47]:
model.train(purchase_train , total_examples=model.corpus_count , epochs=10 , report_delay=1)

(3204396, 3237380)

In [52]:
products = dataset[['StockCode' , 'Description']]

In [60]:
products.drop_duplicates(inplace=True , subset='StockCode' , keep='last')

In [64]:
products_dict = products.groupby('StockCode')['Description'].apply(list).to_dict()

In [65]:
products_dict

{'10002': ['INFLATABLE POLITICAL GLOBE '],
 '10080': ['GROOVY CACTUS INFLATABLE'],
 '10120': ['DOGGY RUBBER'],
 '10123C': ['HEARTS WRAPPING TAPE '],
 '10124A': ['SPOTS ON RED BOOKCOVER TAPE'],
 '10124G': ['ARMY CAMO BOOKCOVER TAPE'],
 '10125': ['MINI FUNKY DESIGN TAPES'],
 '10133': ['COLOURING PENCILS BROWN TUBE'],
 '10135': ['COLOURING PENCILS BROWN TUBE'],
 '11001': ['ASSTD DESIGN RACING CAR PEN'],
 '15030': ['FAN BLACK FRAME '],
 '15034': ['PAPER POCKET TRAVELING FAN '],
 '15036': ['ASSORTED COLOURS SILK FAN'],
 '15039': ['SANDALWOOD FAN'],
 '15044A': ['PINK PAPER PARASOL '],
 '15044B': ['BLUE PAPER PARASOL '],
 '15044C': ['PURPLE PAPER PARASOL'],
 '15044D': ['RED PAPER PARASOL'],
 '15056BL': ['EDWARDIAN PARASOL BLACK'],
 '15056N': ['EDWARDIAN PARASOL NATURAL'],
 '15056P': ['EDWARDIAN PARASOL PINK'],
 '15058A': ['BLUE POLKADOT GARDEN PARASOL'],
 '15058B': ['PINK POLKADOT GARDEN PARASOL'],
 '15058C': ['ICE CREAM DESIGN GARDEN PARASOL'],
 '15060B': ['FAIRY CAKE DESIGN UMBRELLA'],
 '16

In [77]:
def similar_product(v , n=6):
  ms = model.similar_by_vector(v , topn=n+1)[1:]

  new_ms = []
  for j in ms:
    pair = (products_dict[j[0]][0] , j[1])
    new_ms.append(pair)

  return new_ms

In [76]:
similar_product(model['90019A'])

[(['PINK HEART OF GLASS BRACELET'], 0.7652842998504639),
 (['GREEN HEART OF GLASS BRACELET'], 0.7597379684448242),
 (['SILVER M.O.P ORBIT DROP EARRINGS'], 0.7451959848403931),
 (['AMBER DROP EARRINGS W LONG BEADS'], 0.7449609637260437),
 (['SILVER LARIAT BLACK STONE EARRINGS'], 0.7349720001220703),
 (['ANT COPPER RED BOUDICCA BRACELET'], 0.7319890260696411)]

In [70]:
similar_product(model['90019A'])

[('PINK HEART OF GLASS BRACELET', 0.7652842998504639),
 ('GREEN HEART OF GLASS BRACELET', 0.7597379684448242),
 ('SILVER M.O.P ORBIT DROP EARRINGS', 0.7451959848403931),
 ('AMBER DROP EARRINGS W LONG BEADS', 0.7449609637260437),
 ('SILVER LARIAT BLACK STONE EARRINGS', 0.7349720001220703),
 ('ANT COPPER RED BOUDICCA BRACELET', 0.7319890260696411)]

In [78]:
def aggregate_vectors(products):
    product_vec = []
    for i in products:
        try:
            product_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(product_vec, axis=0)

In [81]:
len(purchase_test[0])

16

In [82]:
aggregate_vectors(purchase_test[0]).shape

(100,)

In [84]:
similar_product(aggregate_vectors(purchase_test[0]))

[('HERB MARKER MINT', 0.8734711408615112),
 ('HERB MARKER ROSEMARY', 0.8699325919151306),
 ('HERB MARKER BASIL', 0.86037278175354),
 ('HERB MARKER CHIVES ', 0.8581513166427612),
 ('HERB MARKER PARSLEY', 0.8541859984397888),
 ('BLUE GIANT GARDEN THERMOMETER', 0.6442408561706543)]